In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import matplotlib.ticker as mtick
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
from shapely.geometry import Polygon

import geopandas as gpd
from shapely.geometry import Polygon
from itertools import product
import warnings
warnings.filterwarnings("ignore")

import shapely

In [2]:
chunk_size = 50000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 300000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)
    
# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
zipcode_df = pd.concat(gdf_list, ignore_index=True)

In [3]:
zipcode_df.head()

,ZIPcode,geometry,year
0,99519,"POLYGON ((-148.34155 70.25088, -148.34207 70.2...",2016
1,99519,"POLYGON ((-148.34155 70.25088, -148.34207 70.2...",2015
2,99519,"POLYGON ((-148.34155 70.25088, -148.34207 70.2...",2014
3,99519,"POLYGON ((-148.49985 70.29307, -148.49996 70.2...",2012
4,99519,"POLYGON ((-148.49985 70.29307, -148.49996 70.2...",2013


In [4]:
# Assuming `zipcode_df` is your GeoDataFrame with the shapefiles
#buffered_shapefiles_df = buffer_by_year_iterative(zipcode_df)

In [5]:
import geopandas as gpd
import pandas as pd

def buffer_by_year_binary_search_2(zipcode_df, base_year=2022):
    all_years_buffered = gpd.GeoDataFrame()
    buffer_cache = {}  # Cache for storing last buffer values for each zipcode

    base_year_gdf = zipcode_df[zipcode_df['year'] == base_year]

    for year in range(base_year - 1, 2011, -1):
        print(year)
        df_buffered = base_year_gdf.copy()
        df_buffered['year'] = year

        buffered_geometries = []

        for idx, row in base_year_gdf.iterrows():
            geometry = row['geometry']
            zipcode = row['ZIPcode']

            if geometry is None:
                buffered_geometries.append(None)
                continue

            buffer_percentage = (base_year - year) / 100.0
            target_area = geometry.area * (1 + buffer_percentage * 3)
            initial_guess = buffer_cache.get(zipcode, 0.00001)  # Use cached value if available
            buffer_distance = initial_guess

            buffered_geometry = geometry.buffer(buffer_distance)
            if buffered_geometry.area >= target_area:
                # If initial guess is too large, reduce it to find a lower bound
                high = buffer_distance
                while buffered_geometry.area >= target_area:
                    buffer_distance /= 2
                    buffered_geometry = geometry.buffer(buffer_distance)
                low = buffer_distance
            else:
                # If initial guess is too small, increase it to find an upper bound
                while buffered_geometry.area < target_area:
                    buffer_distance *= 2
                    buffered_geometry = geometry.buffer(buffer_distance)
                low, high = buffer_distance / 2, buffer_distance

            # Apply binary search between low and high
            while high - low > 1e-6:
                mid = (low + high) / 2
                buffered_geometry = geometry.buffer(mid)
                buffered_area = buffered_geometry.area

                if buffered_area < target_area:
                    low = mid
                else:
                    high = mid

            final_buffered_geometry = geometry.buffer((low + high) / 2)
            buffered_geometries.append(final_buffered_geometry)
            buffer_cache[zipcode] = (low + high) / 2  # Update cache

        df_buffered['geometry'] = gpd.GeoSeries(buffered_geometries, index=df_buffered.index)
        all_years_buffered = pd.concat([all_years_buffered, df_buffered], ignore_index=True)

    all_years_buffered = pd.concat([all_years_buffered, base_year_gdf], ignore_index=True)

    return all_years_buffered


In [ ]:
# Convert the geometry column to WKT format
buffered_shapefiles = buffer_by_year_binary_search_2( zipcode_df[zipcode_df['year'] == 2022])

zipcode_shapefiles_12_22 = buffered_shapefiles

2021
2020
2019
2018
2017
2016
2015
2014
2013
2012


In [ ]:
import geopandas as gpd

# Assuming zipcode_df and zipcode_shapefiles_12_22 are already loaded as GeoDataFrame

years = range(2012, 2023)  # From 2012 to 2022, inclusive

# Identify ZIP codes that have an observation in 2022
zipcodes_in_2022 = set(zipcode_df[zipcode_df['year'] == 2022]['ZIPcode'])

average_coverage_per_year = {}

for year in years:
    print(year)
    # Filter rows for the current year and only include ZIP codes with a 2022 observation
    df_year = zipcode_df[(zipcode_df['year'] == year) & (zipcode_df['ZIPcode'].isin(zipcodes_in_2022))]

    coverage_ratios = []

    for _, row in df_year.iterrows():
        # Get corresponding shapefile geometry using ZIPcode
        shapefile_geom = zipcode_shapefiles_12_22[(zipcode_shapefiles_12_22['ZIPcode'] == row['ZIPcode']) & 
                                                  (zipcode_shapefiles_12_22['year'] == year)]['geometry'].iloc[0]

        # Calculate intersection
        intersection = row['geometry'].intersection(shapefile_geom)

        # Calculate area of intersection and area of original geometry
        intersection_area = intersection.area
        original_area = row['geometry'].area

        # Calculate coverage and add to the list
        coverage = intersection_area / original_area if original_area > 0 else 0
        coverage_ratios.append(coverage)

    # Calculate average coverage for the year
    average_coverage = sum(coverage_ratios) / len(coverage_ratios) if coverage_ratios else 0
    average_coverage_per_year[year] = average_coverage

# average_coverage_per_year now contains the average coverage per year
print(average_coverage_per_year)

In [ ]:
import geopandas as gpd

# Assuming zipcode_df is already loaded as a GeoDataFrame

years = range(2012, 2023)  # From 2012 to 2022, inclusive

# Identify ZIP codes that have an observation in 2022
zipcodes_in_2022 = set(zipcode_df[zipcode_df['year'] == 2022]['ZIPcode'])

average_coverage_per_year = {}

for year in years:
    print(year)
    # Filter rows for the current year and only include ZIP codes with a 2022 observation
    df_year = zipcode_df[(zipcode_df['year'] == year) & (zipcode_df['ZIPcode'].isin(zipcodes_in_2022))]

    # Also filter the 2022 geometries
    df_2022 = zipcode_df[zipcode_df['year'] == 2022]

    coverage_ratios = []

    for _, row in df_year.iterrows():
        # Get the corresponding 2022 geometry for the ZIP code
        shapefile_geom_2022 = df_2022[df_2022['ZIPcode'] == row['ZIPcode']]['geometry'].iloc[0]

        # Calculate intersection with the 2022 geometry
        intersection = row['geometry'].intersection(shapefile_geom_2022)

        # Calculate area of intersection and area of original geometry
        intersection_area = intersection.area
        original_area = row['geometry'].area

        # Calculate coverage and add to the list
        coverage = intersection_area / original_area if original_area > 0 else 0
        coverage_ratios.append(coverage)

    # Calculate average coverage for the year
    average_coverage = sum(coverage_ratios) / len(coverage_ratios) if coverage_ratios else 0
    average_coverage_per_year[year] = average_coverage

# average_coverage_per_year now contains the average coverage per year
print(average_coverage_per_year)